<a href="https://colab.research.google.com/github/NayuduPragna/Sentiment-ANALYSIS---movie-reviews/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

Import dependencies


In [3]:
import os
import json
from zipfile import ZipFile
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data from Kaggle

In [7]:
kaggle_dictionary=json.load(open("kaggle.json"))

In [8]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [9]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [10]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


unzip the data file


In [11]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

In [12]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


Loading data

In [13]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [14]:
data.shape

(50000, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [17]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [18]:
data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)

<ipython-input-18-c06d9e4f348c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)


In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [21]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


Splitting data into training data and testing data

In [22]:
train_data, test_data= train_test_split(data, test_size=0.3, random_state=45)

In [23]:
print(train_data.shape)
print(test_data.shape)

(35000, 2)
(15000, 2)


Data preprocessing

In [24]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [25]:
print(X_train)

[[   0    0    0 ...    7    7  448]
 [   0    0    0 ...   37    1    4]
 [   0    0    0 ...    5  417   42]
 ...
 [   5   27   39 ...  534 2467 3269]
 [   0    0    0 ...  436   24 1153]
 [   0    0    0 ...   51 3338 1414]]


In [26]:
print(X_test)

[[   0    0    0 ...   16   54 1138]
 [   0    0    0 ...   18    9 2033]
 [   0    0    0 ...  111  571    9]
 ...
 [   1   19  307 ...   22  536 1154]
 [4203 1253  365 ... 3424  156  156]
 [  11   19   31 ...  103   11   26]]


In [27]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [28]:
print(Y_train)

9855     0
24367    0
43552    0
49483    1
41722    1
        ..
32469    0
16317    0
12895    1
6012     0
6558     0
Name: sentiment, Length: 35000, dtype: int64


In [29]:
print(Y_test)

42462    0
11769    0
32449    1
18688    0
20342    0
        ..
34412    0
23225    0
15728    1
49394    1
44198    0
Name: sentiment, Length: 15000, dtype: int64


LSTM Model

In [31]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [33]:
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [35]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 294s 653ms/step - accuracy: 0.7073 - loss: 0.5478 - val_accuracy: 0.8406 - val_loss: 0.3868
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 312s 638ms/step - accuracy: 0.8640 - loss: 0.3269 - val_accuracy: 0.8406 - val_loss: 0.3768
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 324s 642ms/step - accuracy: 0.8730 - loss: 0.3087 - val_accuracy: 0.8599 - val_loss: 0.3492
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 320s 638ms/step - accuracy: 0.8903 - loss: 0.2754 - val_accuracy: 0.8583 - val_loss: 0.3498
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 321s 636ms/step - accuracy: 0.9095 - loss: 0.2314 - val_accuracy: 0.8614 - val_loss: 0.3499


Model Evaluation

In [36]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

469/469 ━━━━━━━━━━━━━━━━━━━━ 51s 110ms/step - accuracy: 0.8634 - loss: 0.3416
Test Loss: 0.3396647870540619
Test Accuracy: 0.8645333051681519


Building a predictive system

In [37]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

Using Example

In [38]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step
The sentiment of the review is: positive


In [39]:
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
The sentiment of the review is: negative


In [40]:
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
The sentiment of the review is: negative
